In [32]:
import pandas as pd

# Exportamos datos

stats_standard = pd.read_csv('csv/stats_standard.csv')
stats_keeper = pd.read_csv('csv/stats_keeper.csv')
stats_misc = pd.read_csv('csv/stats_misc.csv')
stats_passing = pd.read_csv('csv/stats_passing.csv')
stats_possession = pd.read_csv('csv/stats_possession.csv')
stats_shooting = pd.read_csv('csv/stats_shooting.csv')

salarios = pd.read_csv('csv/salarios.csv')

clasificacion = pd.read_csv('csv/clasificacion.csv')

valor_equipos = pd.read_csv('csv/valorEquipos.csv')

In [2]:
# Refinamos datos

"""
# Convertir datos a float
if(col_float != -1):
    for columna in datos.columns[col_float:]:
        datos[columna] = datos[columna].astype(float)

datos1['Mín'] = datos1['Mín'].str.replace(',', '') #Eliminamos , para poder pasar a float

# Ordenamos los datos
datos_ordenados = datos.sort_values(by=datos.columns[0], ascending=True)
"""

stats_standard = stats_standard[["Jugador", "Posc", "Equipo", "Comp", "Edad", "PJ", "Titular", "Mín",
                                 "Gls.", "Ass", "TP", "TPint", "TA", "TR", "xG", "npxG", "xAG", "PrgC",
                                 "PrgP", "PrgR", "Temporada"]]

stats_keeper = stats_keeper[["Jugador", "Edad", "GC", "DaPC", "Salvadas", "PaC", "TPint", "PD", "PD.1", "PC", "Temporada"]]

stats_misc = stats_misc[["Jugador", "Edad", "2a amarilla", "Fls", "FR", "PA", "Pcz", "Int", "TklG", "Penal concedido",
                         "GC", "Recup.", "Ganados", "Perdidos", "Temporada"]]

stats_shooting = stats_shooting[["Jugador", "Edad", "Dis", "DaP", "Dist", "FK", "Temporada"]]

stats_passing = stats_passing[["Jugador", "Edad", "Cmp", "Int.", "Dist. tot.", "Dist. prg.", "Cmp.1", "Int..1",
                               "Cmp.2", "Int..2", "Cmp.3", "Int..3", "PC", "1/3", "PPA", "CrAP", "Temporada"]]

stats_possession = stats_possession[["Jugador", "Edad", "Toques", "Att", "Succ", "Transportes", "Dist. tot.",
                                     "Dist. prg.", "1/3", "TAP", "Errores de control", "Temporada"]]

salarios = salarios[["Jugador", "Equipo", "Edad", "Salario semanal", "TempAnterior", "Comp"]]

clasificacion = clasificacion[["Equipo", "LgRk", "Temporada"]]

valor_equipos = valor_equipos[['name', 'Total market value', 'Temporada']]

In [3]:
clasificacion.head()

,Equipo,LgRk,Temporada
0,Manchester City,1,2017
1,Juventus,1,2017
2,Bayern Munich,1,2017
3,Paris S-G,1,2017
4,Barcelona,1,2017


In [4]:
# Limpiar las tablas de duplicados (Jugadores que han estado en dos equipos en una temporada)
# Agrupamos los jugadores por su nombre, edad y temporada

# stats_standard
standard = stats_standard.groupby(["Jugador", "Edad", "Temporada"], as_index=False).agg({
    'Posc': 'first',
    'Equipo': 'first',
    'Comp': 'first',
    'PJ': 'sum',
    'Titular': 'sum',
    'Mín': 'sum',
    'Gls.': 'sum',
    'Ass': 'sum',
    'TP': 'sum',
    'TPint': 'sum',
    'TA': 'sum',
    'TR': 'sum',
    'xG': 'sum',
    'npxG': 'sum',
    'xAG': 'sum',
    'PrgC': 'sum',
    'PrgP': 'sum',
    'PrgR': 'sum'
})

# stats_misc
misc = stats_misc.groupby(["Jugador", "Edad", "Temporada"], as_index=False).sum()

# stats_keeper
keeper = stats_keeper.groupby(["Jugador", "Edad", "Temporada"], as_index=False).sum()

# stats_shooting
shooting = stats_shooting.groupby(["Jugador", "Edad", "Temporada"], as_index=False).sum()

# stats_passing
passing = stats_passing.groupby(["Jugador", "Edad", "Temporada"], as_index=False).sum()

# stats_possession
possession = stats_possession.groupby(["Jugador", "Edad", "Temporada"], as_index=False).sum()

# salarios
salary = salarios.groupby(["Jugador", "Edad", "TempAnterior"], as_index=False).agg({
    'Equipo': 'first',
    'Salario semanal': 'first',
    'Comp': 'first'
})

In [5]:
salary[salary["Jugador"]=="João Cancelo"]

,Jugador,Edad,TempAnterior,Equipo,Salario semanal,Comp
7879,João Cancelo,24,2017,Juventus,"€ 106,846 (£ 91,789, $109,510)",Serie-A
7880,João Cancelo,25,2018,Manchester City,"£ 80,000 (€ 95,405, $97,218)",Premier-League
7881,João Cancelo,26,2019,Manchester City,"£ 80,000 (€ 95,405, $97,218)",Premier-League
7882,João Cancelo,27,2020,Manchester City,"£ 80,000 (€ 95,405, $97,218)",Premier-League
7883,João Cancelo,28,2021,Manchester City,"£ 250,000 (€ 290,593, $317,752)",Premier-League
7884,João Cancelo,29,2022,Barcelona,"€ 240,385 (£ 209,033, $253,100)",La-Liga


In [6]:
# Unimos las tablas de datos en una

# Unimos características estandar y porteros
tabla_aux = standard.merge(keeper, on=["Jugador", "Edad", "Temporada"], how='left')
tabla_aux = tabla_aux.fillna(0) # Escribe 0 en los datos faltantes

# Unimos características generales y misc
tabla_aux = tabla_aux.merge(misc, on=["Jugador", "Edad", "Temporada"], how='left')

# Unimos todo con stats de tiros
tabla_aux = tabla_aux.merge(shooting, on=["Jugador", "Edad", "Temporada"], how='left')

# Unimos todo con stats de pases
tabla_aux = tabla_aux.merge(passing, on=["Jugador", "Edad", "Temporada"], how='left')

# Unimos todo con stats de posesion
merged = tabla_aux.merge(possession, on=["Jugador", "Edad", "Temporada"], how='left')


In [7]:
salary.rename(columns={'TempAnterior': 'Temporada'}, inplace=True)

In [8]:
merged['Edad'] = merged['Edad'].astype(int)
merged.head()

,Jugador,Edad,Temporada,Posc,Equipo,Comp,PJ,Titular,Mín,Gls.,...,CrAP,Toques,Att,Succ,Transportes,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control
0,Aaron Connolly,19,2019,DL,Brighton,eng Premier League,24,14,1258,3,...,0.0,318.0,20.0,6.0,198.0,1150.0,417.0,7.0,12.0,53.0
1,Aaron Connolly,20,2020,DL,Brighton,eng Premier League,17,9,791,2,...,0.0,174.0,11.0,5.0,106.0,652.0,276.0,11.0,5.0,27.0
2,Aaron Connolly,21,2021,"DL,CC",Brighton,eng Premier League,4,1,156,0,...,0.0,33.0,2.0,0.0,22.0,121.0,38.0,0.0,0.0,2.0
3,Aaron Cresswell,27,2017,DF,West Ham,eng Premier League,36,35,3069,1,...,10.0,2035.0,14.0,5.0,870.0,4032.0,1959.0,48.0,1.0,26.0
4,Aaron Cresswell,28,2018,DF,West Ham,eng Premier League,20,18,1589,0,...,8.0,1279.0,13.0,6.0,563.0,2544.0,1349.0,25.0,6.0,13.0


In [9]:
filas_aaron_connolly = salary.loc[salary['Jugador'] == 'Aaron Connolly']
filas_aaron_connolly

,Jugador,Edad,Temporada,Equipo,Salario semanal,Comp
0,Aaron Connolly,19,2018,Brighton,"£ 7,500 (€ 8,944, $9,114)",Premier-League
1,Aaron Connolly,20,2019,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League
2,Aaron Connolly,21,2020,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League


Hay que restar uno a la edad, para tener al igual que con la temporada, la edad del año anterior.

In [10]:
salary['Edad'] = salary['Edad'] - 1


In [11]:
filas_aaron_connolly = salary.loc[salary['Jugador'] == 'Aaron Connolly']
filas_aaron_connolly

,Jugador,Edad,Temporada,Equipo,Salario semanal,Comp
0,Aaron Connolly,18,2018,Brighton,"£ 7,500 (€ 8,944, $9,114)",Premier-League
1,Aaron Connolly,19,2019,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League
2,Aaron Connolly,20,2020,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League


In [12]:
# Unimos las tablas de salarios en una y la juntamos con la de datos

# Combina los DataFrames usando los tres identificadores como claves
result = merged.merge(salary, how='outer', on=['Jugador', 'Edad', 'Temporada'])

# Eliminamos filas que no tengan un valor en el salario(NaN) o en las estadisticas de merged, elijo Posc por ejemplo
result.dropna(subset=['Salario semanal'], inplace=True)
result.dropna(subset=['Posc'], inplace=True)

In [13]:
result

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Succ,Transportes,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y
0,Aaron Connolly,19,2019,DL,Brighton,eng Premier League,24.0,14.0,1258.0,3.0,...,6.0,198.0,1150.0,417.0,7.0,12.0,53.0,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League
1,Aaron Connolly,20,2020,DL,Brighton,eng Premier League,17.0,9.0,791.0,2.0,...,5.0,106.0,652.0,276.0,11.0,5.0,27.0,Brighton,"£ 23,077 (€ 27,521, $28,044)",Premier-League
3,Aaron Cresswell,27,2017,DF,West Ham,eng Premier League,36.0,35.0,3069.0,1.0,...,5.0,870.0,4032.0,1959.0,48.0,1.0,26.0,West Ham,"£ 50,000 (€ 58,210, $59,653)",Premier-League
4,Aaron Cresswell,28,2018,DF,West Ham,eng Premier League,20.0,18.0,1589.0,0.0,...,6.0,563.0,2544.0,1349.0,25.0,6.0,13.0,West Ham,"£ 50,000 (€ 59,628, $60,761)",Premier-League
5,Aaron Cresswell,29,2019,DF,West Ham,eng Premier League,31.0,31.0,2727.0,3.0,...,11.0,934.0,3750.0,1808.0,33.0,2.0,18.0,West Ham,"£ 50,000 (€ 59,628, $60,761)",Premier-League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,Ștefan Radu,30,2017,DF,Lazio,it Serie A,31.0,31.0,2588.0,0.0,...,23.0,1251.0,7085.0,4447.0,30.0,2.0,18.0,Lazio,"€ 49,865 (£ 42,838, $51,109)",Serie-A
15946,Ștefan Radu,31,2018,DF,Lazio,it Serie A,28.0,28.0,2243.0,0.0,...,7.0,829.0,4770.0,3112.0,33.0,3.0,21.0,Lazio,"€ 49,808 (£ 41,764, $50,760)",Serie-A
15947,Ștefan Radu,32,2019,DF,Lazio,it Serie A,29.0,28.0,2306.0,1.0,...,18.0,877.0,5202.0,3413.0,33.0,1.0,11.0,Lazio,"€ 49,808 (£ 41,764, $50,760)",Serie-A
15948,Ștefan Radu,33,2020,DF,Lazio,it Serie A,31.0,30.0,2458.0,0.0,...,13.0,1087.0,5724.0,3181.0,38.0,1.0,17.0,Lazio,"€ 42,692 (£ 35,798, $43,508)",Serie-A


In [14]:
# Modificamos algunas columnas como Comp, la cual pone 'it Serie A' por ejemplo
# Ejecutar varias veces consecutivas esta celda causa la pérdida de información en la variable 'Comp'

result['Comp_y'] = result['Comp_y'].str.replace('-',' ')
result['Comp_x'] = result['Comp_x'].str.extract(r'\s(.+$)')
result['Salario semanal'] = result['Salario semanal'].str.extract(r'€ (\d+(?:,\d{3})*(?:\.\d+)?)')

# Convertimos la última columna en float que seguía siendo una cadena de caracteres
result['Salario semanal'] = result['Salario semanal'].str.replace(',', '').astype(float)

In [15]:
result

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Succ,Transportes,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y
0,Aaron Connolly,19,2019,DL,Brighton,Premier League,24.0,14.0,1258.0,3.0,...,6.0,198.0,1150.0,417.0,7.0,12.0,53.0,Brighton,27521.0,Premier League
1,Aaron Connolly,20,2020,DL,Brighton,Premier League,17.0,9.0,791.0,2.0,...,5.0,106.0,652.0,276.0,11.0,5.0,27.0,Brighton,27521.0,Premier League
3,Aaron Cresswell,27,2017,DF,West Ham,Premier League,36.0,35.0,3069.0,1.0,...,5.0,870.0,4032.0,1959.0,48.0,1.0,26.0,West Ham,58210.0,Premier League
4,Aaron Cresswell,28,2018,DF,West Ham,Premier League,20.0,18.0,1589.0,0.0,...,6.0,563.0,2544.0,1349.0,25.0,6.0,13.0,West Ham,59628.0,Premier League
5,Aaron Cresswell,29,2019,DF,West Ham,Premier League,31.0,31.0,2727.0,3.0,...,11.0,934.0,3750.0,1808.0,33.0,2.0,18.0,West Ham,59628.0,Premier League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,Ștefan Radu,30,2017,DF,Lazio,Serie A,31.0,31.0,2588.0,0.0,...,23.0,1251.0,7085.0,4447.0,30.0,2.0,18.0,Lazio,49865.0,Serie A
15946,Ștefan Radu,31,2018,DF,Lazio,Serie A,28.0,28.0,2243.0,0.0,...,7.0,829.0,4770.0,3112.0,33.0,3.0,21.0,Lazio,49808.0,Serie A
15947,Ștefan Radu,32,2019,DF,Lazio,Serie A,29.0,28.0,2306.0,1.0,...,18.0,877.0,5202.0,3413.0,33.0,1.0,11.0,Lazio,49808.0,Serie A
15948,Ștefan Radu,33,2020,DF,Lazio,Serie A,31.0,30.0,2458.0,0.0,...,13.0,1087.0,5724.0,3181.0,38.0,1.0,17.0,Lazio,42692.0,Serie A


In [16]:
clasificacion

,Equipo,LgRk,Temporada
0,Manchester City,1,2017
1,Juventus,1,2017
2,Bayern Munich,1,2017
3,Paris S-G,1,2017
4,Barcelona,1,2017
...,...,...,...
583,Southampton,20,2022
584,Elche,20,2022
585,Troyes,19,2022
586,Sampdoria,20,2022


In [17]:
# Añadimos la información de clasificación a la tabla datos

# Renombramos y creamos nuevas columnas
result.rename(columns={'Equipo_x': 'Equipo'}, inplace=True)

# Combina los DataFrames usando los dos identificadores como claves
result = result.merge(clasificacion, how='outer', on=['Equipo', 'Temporada'])

# Eliminamos filas que no tengan un valor en el LgRk(NaN) o en las estadisticas de merged, elijo Posc por ejemplo
result.dropna(subset=['LgRk'], inplace=True)
result.dropna(subset=['Posc'], inplace=True)

# Devolvemos el nombre antiguo
result.rename(columns={'Equipo': 'Equipo_x'}, inplace=True)

result.head()

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Transportes,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y,LgRk
0,Aaron Connolly,19.0,2019,DL,Brighton,Premier League,24.0,14.0,1258.0,3.0,...,198.0,1150.0,417.0,7.0,12.0,53.0,Brighton,27521.0,Premier League,15
1,Adam Webster,24.0,2019,DF,Brighton,Premier League,31.0,31.0,2668.0,3.0,...,1264.0,7893.0,5013.0,23.0,3.0,19.0,Brighton,41740.0,Premier League,15
2,Alexis Mac Allister,20.0,2019,"CC,DL",Brighton,Premier League,9.0,4.0,352.0,0.0,...,73.0,278.0,115.0,3.0,1.0,7.0,Brighton,36694.0,Premier League,15
3,Alireza Jahanbakhsh,25.0,2019,"DL,CC",Brighton,Premier League,10.0,3.0,311.0,2.0,...,118.0,560.0,252.0,5.0,1.0,8.0,Brighton,59628.0,Premier League,15
4,Bernardo,24.0,2019,"DF,CC",Brighton,Premier League,14.0,7.0,679.0,0.0,...,238.0,1277.0,766.0,6.0,1.0,16.0,Brighton,29814.0,Premier League,15


In [18]:
# Añadimos la información de clasificación a la tabla datos (Equipo_y)

# Renombramos y creamos nuevas columnas
result.rename(columns={'Equipo_y': 'Equipo'}, inplace=True)

# Combina los DataFrames usando los dos identificadores como claves
result = result.merge(clasificacion, how='outer', on=['Equipo', 'Temporada'])

# Eliminamos filas que no tengan un valor en el LgRk(NaN) o en las estadisticas de merged, elijo Posc por ejemplo
result.dropna(subset=['LgRk_y'], inplace=True)
result.dropna(subset=['Posc'], inplace=True)

# Devolvemos el nombre antiguo
result.rename(columns={'Equipo': 'Equipo_y'}, inplace=True)

result.head()

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y,LgRk_x,LgRk_y
0,Aaron Connolly,19.0,2019,DL,Brighton,Premier League,24.0,14.0,1258.0,3.0,...,1150.0,417.0,7.0,12.0,53.0,Brighton,27521.0,Premier League,15.0,15.0
1,Adam Webster,24.0,2019,DF,Brighton,Premier League,31.0,31.0,2668.0,3.0,...,7893.0,5013.0,23.0,3.0,19.0,Brighton,41740.0,Premier League,15.0,15.0
2,Alexis Mac Allister,20.0,2019,"CC,DL",Brighton,Premier League,9.0,4.0,352.0,0.0,...,278.0,115.0,3.0,1.0,7.0,Brighton,36694.0,Premier League,15.0,15.0
3,Alireza Jahanbakhsh,25.0,2019,"DL,CC",Brighton,Premier League,10.0,3.0,311.0,2.0,...,560.0,252.0,5.0,1.0,8.0,Brighton,59628.0,Premier League,15.0,15.0
4,Bernardo,24.0,2019,"DF,CC",Brighton,Premier League,14.0,7.0,679.0,0.0,...,1277.0,766.0,6.0,1.0,16.0,Brighton,29814.0,Premier League,15.0,15.0


In [19]:
# Eliminar jugadores que han jugado muy poco (REVISIÓN)
data_xy = result
#filas_a_eliminar = data_sucio_xy[data_sucio_xy['90 s'] < 5]
#data_xy = data_sucio_xy.drop(filas_a_eliminar.index)

In [20]:
# Convertimos los datos a numéricos
data_xy = data_xy.apply(pd.to_numeric, errors="ignore")

Modificamos el salario semanal ajustándolo a la inflacción anual, las tasas de inflacción de los distintos años las obtengo con el siguiente enlace: https://fxtop.com/es/convertidor-divisas-pasado.php

Usaré los valores de la eurozona, ya que todas las ligas pertenecen a esta zona económica a excepción de la Premier.

In [21]:
# Diccionario con las tasas de inflacción
# 2017 quiere decir temporada 17/18, a la cual le corresponde el salario que cobró el jugador en la 
# siguiente temporada, es decir la 18/19, es decir negociado posiblemente en verano de 2018, aprox.
# Por lo que en 2017 corresponde la tasa de inflacción del 1 de junio de 2018
tasas_inflacion = {
    2017: 0.1829,
    2018: 0.1644,
    2019: 0.1624,
    2020: 0.1466,
    2021: 0.061,
    2022: 0.0
}

In [22]:
# Definir una función que aplica la lógica de inflación a una fila
def aplicar_inflacion(row):
    año = row['Temporada']
    if año in tasas_inflacion:
        inflacion = tasas_inflacion[año]
        return row['Salario semanal'] + row['Salario semanal'] * inflacion
    else:
        return row['Salario semanal']

# Aplicar la función a la columna Salario semanal usando apply
data_xy['Salario semanal'] = data_xy.apply(aplicar_inflacion, axis=1)

# Redondear la columna 'Salario semanal' a 0 decimales
data_xy['Salario semanal'] = data_xy['Salario semanal'].round(0)

In [23]:
data_xy

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y,LgRk_x,LgRk_y
0,Aaron Connolly,19.0,2019,DL,Brighton,Premier League,24.0,14.0,1258.0,3.0,...,1150.0,417.0,7.0,12.0,53.0,Brighton,31990.0,Premier League,15.0,15.0
1,Adam Webster,24.0,2019,DF,Brighton,Premier League,31.0,31.0,2668.0,3.0,...,7893.0,5013.0,23.0,3.0,19.0,Brighton,48519.0,Premier League,15.0,15.0
2,Alexis Mac Allister,20.0,2019,"CC,DL",Brighton,Premier League,9.0,4.0,352.0,0.0,...,278.0,115.0,3.0,1.0,7.0,Brighton,42653.0,Premier League,15.0,15.0
3,Alireza Jahanbakhsh,25.0,2019,"DL,CC",Brighton,Premier League,10.0,3.0,311.0,2.0,...,560.0,252.0,5.0,1.0,8.0,Brighton,69312.0,Premier League,15.0,15.0
4,Bernardo,24.0,2019,"DF,CC",Brighton,Premier League,14.0,7.0,679.0,0.0,...,1277.0,766.0,6.0,1.0,16.0,Brighton,34656.0,Premier League,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,Son Heung-min,25.0,2017,DL,Tottenham,Premier League,37.0,27.0,2317.0,12.0,...,5794.0,2729.0,46.0,68.0,65.0,Tottenham,130828.0,Premier League,3.0,3.0
11218,Toby Alderweireld,28.0,2017,DF,Tottenham,Premier League,14.0,13.0,1178.0,0.0,...,3145.0,2152.0,12.0,0.0,4.0,Tottenham,68857.0,Premier League,3.0,3.0
11219,Victor Wanyama,26.0,2017,CC,Tottenham,Premier League,18.0,8.0,846.0,1.0,...,2387.0,1303.0,17.0,0.0,13.0,Tottenham,89514.0,Premier League,3.0,3.0
11220,Vincent Janssen,23.0,2017,DF,Tottenham,Premier League,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Tottenham,46823.0,Premier League,3.0,3.0


Para las ligas cambiamos el nombre por los beneficios obtenidos por la misma en la temporada que corresponda.

https://www.statista.com/statistics/261218/big-five-european-soccer-leagues-revenue/

In [24]:
# Diccionario de diccionarios con los beneficios de las distintas ligas y temporadas
# 2017 quiere decir temporada 17/18, cantidades expresadas en millones de euros
beneficios_ligas = {
    2017: {'Premier League': 5440, 'Serie A': 2239, 'Bundesliga': 3168, 'La Liga': 3073, 'Ligue 1': 1692},
    2018: {'Premier League': 5843, 'Serie A': 2495, 'Bundesliga': 3345, 'La Liga': 3378, 'Ligue 1': 1902},
    2019: {'Premier League': 5123, 'Serie A': 2057, 'Bundesliga': 3208, 'La Liga': 3114, 'Ligue 1': 1598},
    2020: {'Premier League': 5498, 'Serie A': 2521, 'Bundesliga': 3005, 'La Liga': 2953, 'Ligue 1': 1614},
    2021: {'Premier League': 6442, 'Serie A': 2350, 'Bundesliga': 3149, 'La Liga': 3277, 'Ligue 1': 2026},
    2022: {'Premier League': 6605, 'Serie A': 2600, 'Bundesliga': 3550, 'La Liga': 3300, 'Ligue 1': 2100}
}

In [25]:
# Codificamos las ligas con su beneficio
def codifica_liga_x(row):
    año = row['Temporada']
    liga = row['Comp_x']

    if año in beneficios_ligas and liga in beneficios_ligas[año]:
        return beneficios_ligas[año][liga]
    else:
        print('ERROR')
        return liga
    
def codifica_liga_y(row):
    año = row['Temporada']
    liga = row['Comp_y']

    if año in beneficios_ligas and liga in beneficios_ligas[año]:
        return beneficios_ligas[año][liga]
    else:
        print('ERROR')
        return liga

# Aplicar las funciones a las columnas usando apply
data_xy['Comp_x'] = data_xy.apply(codifica_liga_x, axis=1)
data_xy['Comp_y'] = data_xy.apply(codifica_liga_y, axis=1)

data_xy

,Jugador,Edad,Temporada,Posc,Equipo_x,Comp_x,PJ,Titular,Mín,Gls.,...,Dist. tot._y,Dist. prg._y,1/3_y,TAP,Errores de control,Equipo_y,Salario semanal,Comp_y,LgRk_x,LgRk_y
0,Aaron Connolly,19.0,2019,DL,Brighton,5123,24.0,14.0,1258.0,3.0,...,1150.0,417.0,7.0,12.0,53.0,Brighton,31990.0,5123,15.0,15.0
1,Adam Webster,24.0,2019,DF,Brighton,5123,31.0,31.0,2668.0,3.0,...,7893.0,5013.0,23.0,3.0,19.0,Brighton,48519.0,5123,15.0,15.0
2,Alexis Mac Allister,20.0,2019,"CC,DL",Brighton,5123,9.0,4.0,352.0,0.0,...,278.0,115.0,3.0,1.0,7.0,Brighton,42653.0,5123,15.0,15.0
3,Alireza Jahanbakhsh,25.0,2019,"DL,CC",Brighton,5123,10.0,3.0,311.0,2.0,...,560.0,252.0,5.0,1.0,8.0,Brighton,69312.0,5123,15.0,15.0
4,Bernardo,24.0,2019,"DF,CC",Brighton,5123,14.0,7.0,679.0,0.0,...,1277.0,766.0,6.0,1.0,16.0,Brighton,34656.0,5123,15.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11217,Son Heung-min,25.0,2017,DL,Tottenham,5440,37.0,27.0,2317.0,12.0,...,5794.0,2729.0,46.0,68.0,65.0,Tottenham,130828.0,5440,3.0,3.0
11218,Toby Alderweireld,28.0,2017,DF,Tottenham,5440,14.0,13.0,1178.0,0.0,...,3145.0,2152.0,12.0,0.0,4.0,Tottenham,68857.0,5440,3.0,3.0
11219,Victor Wanyama,26.0,2017,CC,Tottenham,5440,18.0,8.0,846.0,1.0,...,2387.0,1303.0,17.0,0.0,13.0,Tottenham,89514.0,5440,3.0,3.0
11220,Vincent Janssen,23.0,2017,DF,Tottenham,5440,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Tottenham,46823.0,5440,3.0,3.0


Ahora debemos codificar los equipos con su valor de plantilla. Primero limpiamos la tabla valorEquipos.

In [27]:
valor_equipos

,name,Total market value,Temporada
0,Manchester City,€1010m,2017
1,Chelsea FC,€884.25m,2017
2,Liverpool FC,€857.50m,2017
3,Manchester United,€849.50m,2017
4,Tottenham Hotspur,€829.60m,2017
...,...,...,...
583,RCD Mallorca,€107.00m,2022
584,Rayo Vallecano,€103.40m,2022
585,Real Valladolid CF,€90.75m,2022
586,Elche CF,€67.50m,2022


Modificamos los total market value para que solo aparezca la cifra.

In [34]:
# Eliminar el símbolo de euro y la "m"
valor_equipos['Total market value'] = valor_equipos['Total market value'].str.replace('€', '').str.replace('m', '')
valor_equipos['Total market value'] = valor_equipos['Total market value'].astype(float)

valor_equipos

,name,Total market value,Temporada
0,Manchester City,1010.00,2017
1,Chelsea FC,884.25,2017
2,Liverpool FC,857.50,2017
3,Manchester United,849.50,2017
4,Tottenham Hotspur,829.60,2017
...,...,...,...
583,RCD Mallorca,107.00,2022
584,Rayo Vallecano,103.40,2022
585,Real Valladolid CF,90.75,2022
586,Elche CF,67.50,2022


Cambiamos los nombres a las columnas para que sean más informativos.

In [4]:
data_xy.columns

Index(['Jugador', 'Edad', 'Temporada', 'Posc', 'Comp_x', 'PJ', 'Titular',
       'Mín', 'Gls.', 'Ass', 'TP', 'TPint_x', 'TA', 'TR', 'xG', 'npxG', 'xAG',
       'PrgC', 'PrgP', 'PrgR', 'GC_x', 'DaPC', 'Salvadas', 'PaC', 'TPint_y',
       'PD', 'PD.1', 'PC_x', '2a amarilla', 'Fls', 'FR', 'PA', 'Pcz', 'Int',
       'TklG', 'Penal concedido', 'GC_y', 'Recup.', 'Ganados', 'Perdidos',
       'Dis', 'DaP', 'Dist', 'FK', 'Cmp', 'Int.', 'Dist. tot._x',
       'Dist. prg._x', 'Cmp.1', 'Int..1', 'Cmp.2', 'Int..2', 'Cmp.3', 'Int..3',
       'PC_y', '1/3_x', 'PPA', 'CrAP', 'Toques', 'Att', 'Succ', 'Transportes',
       'Dist. tot._y', 'Dist. prg._y', '1/3_y', 'TAP', 'Errores de control',
       'Salario semanal', 'Comp_y', 'LgRk_x', 'LgRk_y'],
      dtype='object')

In [ ]:
nuevos_nombres = {'Posc': 'Posicion', 'B': 'Columna2'}
data_xy = data_xy.rename(columns=nuevos_nombres)
data_xy.columns

In [73]:
# Exportamos la tabla final

ruta_archivo_csv = 'csv/DataFinal.csv'

# Exporta el DataFrame a un archivo CSV
data_xy.to_csv(ruta_archivo_csv, index=False)